In [1]:

from PIL import Image
import torch
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import numpy as np
from network import Unsupervised_Object_Detection
import matplotlib.pyplot as plt
from utils import disp_activation, get_activation

im = Image.open('bottle1.jpg')
target_list = {
    'boxes': torch.tensor([[463.0, 20.0, 666.0, 588.0]], dtype=torch.float32),  # Ensure float32 type
    'labels': torch.tensor([1], dtype=torch.int64),  # Class label should be int64
    # 'image_id': torch.tensor([1], dtype=torch.int64)  # Unique ID should be int64
}
im_test = Image.open('bottle2.jpg')
model = Unsupervised_Object_Detection()
# zi, zj, za = model(im)


# Train

In [2]:
from utils import random_rotate_image, random_augment
from loss_func import nt_xent_loss

# Pre-training function
def pre_train_one_image(model, image, optimizer,scheduler,loss, epochs=10):
    
    new_im = image
    model.train()
    for epoch in range(epochs):
        zi, zj, za = model(new_im)
        loss = nt_xent_loss(zi,zj,za)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        print(f'Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}')
        # new_im = random_rotate_image(image)
        new_im = random_augment(image)
    
    scheduler.step()

loss = nt_xent_loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
pre_train_one_image(model,im,optimizer,scheduler, loss)

/Users/erezshani/Projects/Unsupervised_costractive/pipelines.py:123: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525498485/work/torch/csrc/utils/tensor_new.cpp:233.)
  return torch.tensor(anchor_negatives, device=fpn_output.device)


Epoch 1/10, Loss: 2.15993595123291
Epoch 2/10, Loss: 1.2067207098007202
Epoch 3/10, Loss: 1.0558346509933472
Epoch 4/10, Loss: 0.11328437179327011
Epoch 5/10, Loss: 0.11015941947698593
Epoch 6/10, Loss: 0.08154162764549255
Epoch 7/10, Loss: 0.06114836782217026
Epoch 8/10, Loss: 0.25982779264450073
Epoch 9/10, Loss: 0.017796523869037628
Epoch 10/10, Loss: 0.01975332200527191


Fine Tune

In [ ]:
from fine_tune_utiles import random_augment_ft, plot_image_with_boxes
target_list = {
    'boxes': torch.tensor([[463.0, 20.0, 666.0, 588.0]], dtype=torch.float32),  # Ensure float32 type
    'labels': torch.tensor([1], dtype=torch.int64),  # Class label should be int64
    # 'image_id': torch.tensor([1], dtype=torch.int64)  # Unique ID should be int64
}
new_im, new_target = random_augment_ft(im, target_list)
plot_image_with_boxes(new_im, new_target['boxes'])


In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        """
        Focal Loss for binary classification.
        
        :param alpha: Balancing factor (usually between 0.25 and 1).
        :param gamma: Focusing parameter (usually 2).
        :param reduction: Specifies the reduction to apply to the output: 'none' | 'mean' | 'sum'.
        """
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        """
        Forward pass of the loss function.
        
        :param inputs: Predicted logits (before sigmoid), shape (N, *).
        :param targets: Ground truth binary labels (0 or 1), shape (N, *).
        :return: Loss value.
        """
        # Convert targets to the same shape as inputs
        targets = targets.view(-1, 1)
        inputs= inputs.view(-1, 1)
        
        # Apply sigmoid to get the probabilities
        BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        
        # Get probabilities after sigmoid
        pt = torch.exp(-BCE_loss)
        
        # Calculate focal loss
        F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
        
        if self.reduction == 'mean':
            return F_loss.mean()
        elif self.reduction == 'sum':
            return F_loss.sum()
        else:
            return F_loss

In [7]:
from fine_tune_utiles import random_augment_ft

focal_loss = FocalLoss(alpha=1, gamma=2)
def fine_tune_retinanet(model, images, targets):

    model.train()
    # Freeze the backbone
    for param in model.backbone.parameters():
        param.requires_grad = False
    # Train head
    for param in model.head.parameters():
        param.requires_grad = True

    # optimizer = torch.optim.Adam([param for param in model.parameters() if param.requires_grad], lr=0.0001)
    optimizer = torch.optim.SGD(
        [param for param in model.parameters() if param.requires_grad],
        lr=0.001,
        momentum=0.8,
        weight_decay=0.00001,
    )
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
    
    num_epochs = 8

    
    new_im =  transforms.ToTensor()(images)
    new_target = targets
    for epoch in range(num_epochs):
        # new_im = [new_im]#.to('cuda')
        new_im = [new_im]
        targets = [{k: v for k, v in new_target.items()}]

        # Forward pass
        outputs = model(new_im, targets)  # Get model outputs (logits)

        classification_loss = focal_loss(outputs['classification'], targets[0]['labels'].squeeze())
        regression_loss = outputs['bbox_regression'].loss(targets['boxes'])  # Assuming bbox regression loss
        
                # Combine the losses
        losses = classification_loss + regression_loss

        # # Forward pass
        # loss_dict = model(new_im, targets)
        # losses = sum(loss for loss in loss_dict.values())

        # Backward pass
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        # new_im = random_augment(images)
        new_im, new_target = random_augment_ft(images, target_list)
        new_im = transforms.ToTensor()(new_im)

        print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {losses.item():.4f}")
    scheduler.step()
    

fine_tune_retinanet(model.P2.retinanet,im,target_list)

RuntimeError: result type Float can't be cast to the desired output type Long

In [ ]:
# torch.save(model.state_dict(), 'model.pth')

model = Unsupervised_Object_Detection()
model.load_state_dict(torch.load('model.pth'))
model.eval()
print()

Plot

In [ ]:
from fine_tune_utiles import plot_predictions

# Example usage
img = transforms.ToTensor()(im)
img = img.unsqueeze(0) # needs to be 4d!
model.P2.retinanet.eval()
predictions = model.P2.retinanet(img)
# print(predictions[0]['scores'])
plot_predictions(im, predictions)


In [ ]:
# Example usage
img = transforms.ToTensor()(im_test)
img = img.unsqueeze(0) # needs to be 4d!
model.P2.retinanet.eval()
predictions = model.P2.retinanet(img)
# print(predictions[0]['scores'])
plot_predictions(im_test, predictions,threshold=0.2)

<hr>

In [ ]:
def min_max(x):
    return (x - np.min(x))/(np.max(x) - np.min(x))

def get_activation(activation, img):
    # w,h = img.size
    h,w = img.size

    # Normalize the Grad-CAM
    grad_cam = torch.nn.functional.relu(activation)
    grad_cam = torch.nn.functional.interpolate(grad_cam.unsqueeze(0), size=(w,h), mode='bilinear', align_corners=False)
    grad_cam = grad_cam.detach().squeeze().cpu().numpy()
    grad_cam = (grad_cam - grad_cam.min()) / (grad_cam.max() - grad_cam.min())
    return grad_cam

w,h = im.size
total_activation = np.zeros(shape=(h,w))
tensor_im = transforms.ToTensor()(im)
output = model.P2.retinanet.backbone(tensor_im)
for i, (name,data) in enumerate(output.items()):
    total_activation += get_activation(data.mean(1), im)
total_activation = min_max(total_activation)




In [ ]:
tranpose_im = np.array(im.transpose(Image.FLIP_LEFT_RIGHT))
plt.imshow(tranpose_im)
mask = np.stack([total_activation] * 3, axis=-1) 
a = mask * tranpose_im
a = np.clip(a, 0, 255).astype(np.uint8)
# plt.imshow(a)
plt.imshow(mask)

In [ ]:
np.max(total_activation)

In [ ]:
plt.imshow(total_activation>0.7)

In [ ]:
from utils import disp_activation, get_activation

tensor_im = transforms.ToTensor()(im)
output = model.P2.retinanet.backbone(tensor_im)
N_images = len(output)
fig, ax = plt.subplots(1,N_images, figsize = (12,12))
for i, (name,data) in enumerate(output.items()):
    print(name)
    disp_activation(data.mean(1), im,ax[i], name)

In [ ]:
tensor_im = transforms.ToTensor()(im_test)
output = model.P2.retinanet.backbone(tensor_im)
N_images = len(output)
fig, ax = plt.subplots(1,N_images, figsize = (12,12))
for i, (name,data) in enumerate(output.items()):
    print(name)
    disp_activation(data.mean(1), im_test,ax[i], name)

In [ ]:
from testing_functions import livecam_test
from capture_box_from_image import draw_rectangle